In [1]:
import glob
import whisper
import os 
import json
from openai import OpenAI
from dotenv import load_dotenv

config = load_dotenv(verbose=False)

### pyaudio

In [1]:
import pyaudio
import wave
import whisper

model = whisper.load_model("medium")

def transcribe_directly():
    sample_rate = 16000
    bits_per_sample = 16
    chunk_size = 1024
    audio_format = pyaudio.paInt16
    channels = 1

    def callback(in_data, frame_count, time_info, status):
        wav_file.writeframes(in_data)
        return None, pyaudio.paContinue

    # Open the wave file for writing
    wav_file = wave.open('./output.wav', 'wb')
    wav_file.setnchannels(channels)
    wav_file.setsampwidth(bits_per_sample // 8)
    wav_file.setframerate(sample_rate)

    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Start recording audio
    stream = audio.open(format=audio_format,
                        channels=channels,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=chunk_size,
                        stream_callback=callback)

    input("Press Enter to stop recording...")
    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Close the wave file
    wav_file.close()
    result = model.transcribe("./output.wav", language="ja")
    return result['text']

In [ ]:
text = transcribe_directly()
print(text)

In [3]:
import pyaudio

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print(f"Device {i}: {dev['name']}")
    print(f"  Input channels: {dev['maxInputChannels']}")
    print(f"  Output channels: {dev['maxOutputChannels']}")
    print(f"  Default Sample Rate: {dev['defaultSampleRate']}")
    print()
p.terminate()

### sounddeivce


In [ ]:
import sounddevice as sd
import numpy as np
import whisper
from scipy.io.wavfile import write

model = whisper.load_model("medium")

def record_audio(duration, samplerate=16000):
    print("Recording...")
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1)
    sd.wait()
    print("Recording finished")
    return recording

def save_audio(filename, recording, samplerate):
    write(filename, samplerate, (recording * 32767).astype(np.int16))

def transcribe_audio(filename, language="ja"):
    result = model.transcribe(filename, language=language)
    return result['text']

def main():
    duration = 5  # Recording duration in seconds
    samplerate = 16000
    filename = "output.wav"

    # Record audio
    recording = record_audio(duration, samplerate)

    # Save audio to file
    save_audio(filename, recording, samplerate)

    # Transcribe audio
    transcription = transcribe_audio(filename)
    print("Transcription:", transcription)

if __name__ == "__main__":
    main()

### whisper

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="Hello world! This is a streaming test.",
)

response.stream_to_file("output.mp3")

In [2]:
wav = sorted(glob.glob1("./data/sample_wav", pattern="*.wav"))
jsons = sorted(glob.glob1("./data/sample_wav", pattern="*.json"))

def test_audio_model(model, n):
    # tiny, base, small, medium, large, large-v2
    url = "./data/sample_wav"
    for idx, (audio, script) in enumerate(zip(wav, jsons)):
        pth = os.path.join(url, audio)
        res_pth = os.path.join(url, script)
        
        audio = whisper.load_audio(pth)
        res = model.transcribe(audio, fp16=False)
        
        with open(res_pth) as f:
            data = json.load(f) 
        
        print(idx+1)
        print('정답: ', data["발화정보"]['stt'])       
        print(f"예측: {res['text']}")
        
        print('-----------------------------------------')
        if idx == n-1:
            break
        

In [17]:
# tiny
model = whisper.load_model("tiny", device='cpu')
test_audio_model(model, 20)

1
정답:  얼마 전에 아파트 인테리어를 새로 했는데 그때 베란다에 방수 시트를 깔았거든요
예측:  얼마 전에 아파트 인테리어를 새로 했는데 그때 배란다의 방수시트를 깔았거든요
-----------------------------------------
2
정답:  (NO:저희) 집은 북동향이라 햇빛이 잘 들어오는 편은 아니에요
예측:  저희 집은 북동영이라 햇빛이 잘 들어오는 편은 아니에요
-----------------------------------------
3
정답:  다들 남향집을 찾는데 희한하게도 햇빛이 강한 집이 싫더라고요
예측:  다들 남형 집을 찾는데, 해야나게 또 햇빛이 강한 집이 싫더라고요.
-----------------------------------------
4
정답:  흙은 그냥 아파트 앞에 있는 걸 퍼다 써도 될까요
예측:  흙은 그냥 아파트 앞에 있는 걸 퍼다 써도 될까요?
-----------------------------------------
5
정답:  왜요 화초는 보통 며칠 주기로 물을 충분히 주는게 좋다고 하던데
예측:  왜요? 하초는 보통 몇 채 주기로 물을 충분히 주는게 좋다고 하었는데?
-----------------------------------------
6
정답:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠 
예측:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠?
-----------------------------------------
7
정답:  밖에서 아이스커피를 먹을 때 이제부터 버리지 말고 집에 가져와야겠네요
예측:  밖에서 아이스크베필을 먹을 때 이제부터 버리지 말고 집에 가져와야겠네요.
-----------------------------------------
8
정답:  그런데 현대에 들어서면서 육류 섭취가 늘어나고 그것 때문에 변이 단단해져서 변비가 생기(SP:기) 시작한 거더라고
예측:  그런데 현대에 들어서 면서 용료 섭취가 늘어나고 그거때문에 변이 단단해져서 변비가 생

In [3]:
# base
model = whisper.load_model("base", device='cpu')
test_audio_model(model, 20)

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 35.6MiB/s]


1
정답:  얼마 전에 아파트 인테리어를 새로 했는데 그때 베란다에 방수 시트를 깔았거든요
예측:  얼마 전에 아파트 인테리어를 새로 했는데 그때 베란다의 방수시트를 깔았거든요.
-----------------------------------------
2
정답:  (NO:저희) 집은 북동향이라 햇빛이 잘 들어오는 편은 아니에요
예측:  저희 집은 북동양이라 햇빛이 잘 들어오는 편은 아니에요.
-----------------------------------------
3
정답:  다들 남향집을 찾는데 희한하게도 햇빛이 강한 집이 싫더라고요
예측:  다들 남양 집을 찾는데, 희한하게도 햇빛이 강한 집이 싫더라고요.
-----------------------------------------
4
정답:  흙은 그냥 아파트 앞에 있는 걸 퍼다 써도 될까요
예측:  흙은 그냥 아파트 앞에 있는 걸 펴다 써도 될까요?
-----------------------------------------
5
정답:  왜요 화초는 보통 며칠 주기로 물을 충분히 주는게 좋다고 하던데
예측:  매야 하초는 보통 며칠 주기로 물을 충분히 주는게 좋다고 하던데
-----------------------------------------
6
정답:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠 
예측:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠.
-----------------------------------------
7
정답:  밖에서 아이스커피를 먹을 때 이제부터 버리지 말고 집에 가져와야겠네요
예측:  밖에서 아이스크비를 먹을게 이제부터 버리지 말고 집에 가져와야겠네요
-----------------------------------------
8
정답:  그런데 현대에 들어서면서 육류 섭취가 늘어나고 그것 때문에 변이 단단해져서 변비가 생기(SP:기) 시작한 거더라고
예측:  그런데 현대에 들어서 면역력 섭취가 늘어나고 그 때문에 변이 단단해져서 변비가 생기기 시작한 거

In [4]:
# small
model = whisper.load_model("small", device='cpu')
test_audio_model(model, 20)

100%|███████████████████████████████████████| 461M/461M [00:15<00:00, 32.1MiB/s]


1
정답:  얼마 전에 아파트 인테리어를 새로 했는데 그때 베란다에 방수 시트를 깔았거든요
예측:  얼마 전에 아파트 인테리어를 새로 했는데 그때 베란다에 방수시트를 깔았거든요.
-----------------------------------------
2
정답:  (NO:저희) 집은 북동향이라 햇빛이 잘 들어오는 편은 아니에요
예측:  저희 집은 북동향이라 햇빛이 잘 들어오는 편은 아니에요.
-----------------------------------------
3
정답:  다들 남향집을 찾는데 희한하게도 햇빛이 강한 집이 싫더라고요
예측:  다들 남형집을 찾는데 희한하게 또 햇빛이 강한 집이 싫더라고요.
-----------------------------------------
4
정답:  흙은 그냥 아파트 앞에 있는 걸 퍼다 써도 될까요
예측:  흙은 그냥 아파트 앞에 있는 걸 퍼다 써도 될까요?
-----------------------------------------
5
정답:  왜요 화초는 보통 며칠 주기로 물을 충분히 주는게 좋다고 하던데
예측:  왜요? 하초는 보통 며칠 주기로 물을 충분히 주는 게 좋다고 하던데
-----------------------------------------
6
정답:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠 
예측:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠?
-----------------------------------------
7
정답:  밖에서 아이스커피를 먹을 때 이제부터 버리지 말고 집에 가져와야겠네요
예측:  밖에서 아이스 커피를 먹을 때 이제부터 버리지 말고 집에 가져와야겠네요.
-----------------------------------------
8
정답:  그런데 현대에 들어서면서 육류 섭취가 늘어나고 그것 때문에 변이 단단해져서 변비가 생기(SP:기) 시작한 거더라고
예측:  그런데 현대에 들어서면서 용류 섭취가 늘어나고 그것 때문에 변이 단단해져서 변비가 생기기

In [5]:
# medium
model = whisper.load_model("medium", device='cpu')
test_audio_model(model, 20)

1
정답:  얼마 전에 아파트 인테리어를 새로 했는데 그때 베란다에 방수 시트를 깔았거든요
예측:  얼마 전에 아파트 인테리어를 새로 했는데 그때 베란다에 방수시트를 깔았거든요.
-----------------------------------------
2
정답:  (NO:저희) 집은 북동향이라 햇빛이 잘 들어오는 편은 아니에요
예측:  저희 집은 북동양이라 햇빛이 잘 들어오는 편은 아니에요
-----------------------------------------
3
정답:  다들 남향집을 찾는데 희한하게도 햇빛이 강한 집이 싫더라고요
예측:  다들 남양집을 찾는데 희한하게도 햇빛이 강한 집이 싫더라구요
-----------------------------------------
4
정답:  흙은 그냥 아파트 앞에 있는 걸 퍼다 써도 될까요
예측:  흙은 그냥 아파트 앞에 있는 걸 퍼다 써도 될까요?
-----------------------------------------
5
정답:  왜요 화초는 보통 며칠 주기로 물을 충분히 주는게 좋다고 하던데
예측:  왜요? 화초는 보통 며칠 주기로 물을 충분히 주는게 좋다고 하던데
-----------------------------------------
6
정답:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠 
예측:  화분을 사려고 했는데 처음부터 그럴 필요 없는 거죠?
-----------------------------------------
7
정답:  밖에서 아이스커피를 먹을 때 이제부터 버리지 말고 집에 가져와야겠네요
예측:  밖에서 아이스 커피를 먹을 때 이제부터 버리지 말고 집에 가져와야겠네요
-----------------------------------------
8
정답:  그런데 현대에 들어서면서 육류 섭취가 늘어나고 그것 때문에 변이 단단해져서 변비가 생기(SP:기) 시작한 거더라고
예측:  그런데 현대에 들어서면서 용유 섭취가 늘어나고 그것 때문에 변이 단단해져서 변비가 생기기 시작한 

In [ ]:
# large
model = whisper.load_model("large", device='cpu')
test_audio_model(model, 20)

In [ ]:
# large-v2
model = whisper.load_model("large-v2", device='cpu')
test_audio_model(model, 20)